In [4]:
import pandas as pd
import scipy.io as sio
import numpy as np
import datetime
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import re

def Array_To_DataFrame(matDF, varName) :
    npArray = np.array([])
    for i in range (matDF[varName].shape[0]) :
        npArray = np.concatenate((npArray, matDF[varName][i].flatten()))
    return npArray
def matFile_To_DF(matFile) :
    tempDF = sio.loadmat(matFile)
    matDataMidFiltered = {dictKey:dictValue for dictKey, dictValue in tempDF.items() if dictKey[0] != '_'}
    matMidDF = pd.DataFrame({dictKey: np.array(dictValue).flatten() for dictKey, dictValue in matDataMidFiltered.items()})
    return matMidDF

matWeather = sio.loadmat('wx_2013_2014.mat')
origin = np.datetime64('0000-01-01', 'D') - np.timedelta64(1, 'D')
matWeatherFiltered = {dictKey:dictValue for dictKey, dictValue in matWeather.items() if dictKey[0] != '_'}
matWeatherDF = pd.DataFrame({dictKey: np.array(dictValue).flatten() for dictKey, dictValue in matWeatherFiltered.items()})
tmsWeather = matWeatherDF['dnwx'] * np.timedelta64(1, 'D') + origin
tmsAirTemp = matWeatherDF['t']
matWeatherDF = matWeatherDF.sort_values(['dnwx'], ascending = [True])

weatherDF = pd.DataFrame({'DN' : tmsWeather, 'AirTemp' : tmsAirTemp})
weatherDF.to_csv('weather_2013_2014_processed.csv', index=False)

tempFigLabels = ['Air Temperature']
tempfig = px.line(x = weatherDF['DN'], y = [weatherDF["AirTemp"]], color_discrete_sequence= ["black"], title = "Air Temperature")
for idx in range(len(tempFigLabels)):
    tempfig.data[idx].name = tempFigLabels[idx]
    tempfig.data[idx].hovertemplate = 'variable=' + tempFigLabels[idx] + '<br>x=%{x}<br>value=%{y}<extra></extra>'
    tempfig.data[idx].legendgroup = tempFigLabels[idx]

tempfig.update_layout(title=dict(text= "Air Temperature for 2013-2014", font=dict(size=25)))
tempfig.update_xaxes(tickangle=30)
tempfig.update_yaxes(range = (-5,30))
tempfig.update_xaxes(rangeslider_visible=True)
tempfig.update_xaxes(range = [pd.Timestamp('2013-10-22'),pd.Timestamp('2014-03-15')])
tempfig.update_layout(xaxis_title="Date", yaxis_title="Temperature (°C)")
tempfig.show()


